In [21]:
"""
This notebook is the main part of the bachelors project 'A concept and prototypical implementation for network based 
analysis of fish behavior' by Nicolai Kraus at the University of Constance, supported by Michael Aichem and 
supervised by Dr. Karsten Klein. 

The project consists of a pipeline which loads a behavior dataset and produces a interactive dashboard via jupyter 
notebook and voila.

Usage: Read "README.md", install needed libraries and load the dashboard via jupyter 
notebook with the button in the upper right-hand corner or via 'voila analyse_behavior.ipynb' in the command line.

"""
import networkx as nx
from networkx.drawing.nx_pydot import to_pydot
import matplotlib.pyplot as plt
import pandas as pd
import math
import os
import numpy as np
import io
import graphviz
from graphviz import Source
import matplotlib as mpl
from ipywidgets import interactive,interact, fixed
import ipywidgets as widgets
from IPython.display import display, Image, Markdown
pd.options.mode.chained_assignment = None  # default='warn'

#this line is needed for windows so the library 'pygraphviz', a wrapper of 'graphviz' for 'python'
#can load its modules 'dot' and 'neato' properly.
if  not 'C:\\Program Files (x86)\\Graphviz2.38\\bin' in os.environ["PATH"]: 
    os.environ["PATH"] += os.pathsep + 'C:\\Program Files (x86)\\Graphviz2.38\\bin'  

def get_fish_ids(boris_df):
    """This function collects the unique fish ids. Functions needing these call this function so the order
    is always the same which results in a consequent colour scheme over all plots."""
    fish_ids = boris_df.subject.unique().tolist()
    if 'Subject' in fish_ids: fish_ids.remove('Subject')
    fish_ids = [x for x in fish_ids if str(x) != 'nan']
    return fish_ids

def create_interaction_network(boris_df, min_interactions=1):
    """This function takes as parameters the behavior file and the minimum number of interactions between two fish 
    for an edge to be displayed. An edge count is increased for each row in the behavior file where 'subject' 
    and 'modifier_1' are the same."""
    #remove behavior with no interaction partner and irrelevant data
    interactions_df = boris_df[boris_df.modifier_1.notna()]
    interactions_df = interactions_df[['subject', 'modifier_1']]
    #create a dataframe for the edges 
    edges_df = interactions_df.groupby(interactions_df.columns.tolist(), as_index=False).size().to_frame(name='records').reset_index()
    #remove edges below the threshold
    edges_df = edges_df[edges_df.records >= min_interactions]
    #add tuples and records as attributes for the network generation
    edges_df['tuples'] = list(zip(edges_df.subject, edges_df.modifier_1))
    edge_attributes_label = dict(zip(edges_df.tuples, edges_df.records))
    #change for edge weight
    edges_df.records = edges_df.records * 3 / edges_df.records.max()
    edge_attributes_weight = dict(zip(edges_df.tuples, edges_df.records))
    #create directed graph with networkx
    G = nx.DiGraph()
    G.add_edges_from(edges_df.tuples)
    #edge labels
    nx.set_edge_attributes(G, edge_attributes_label, name='label')
    #edge weight
    nx.set_edge_attributes(G, edge_attributes_weight, name='penwidth')
    #graphviz
    G_dot_string = to_pydot(G).to_string()
    G_dot = Source(G_dot_string)
    G_dot.format= 'png'
    G_dot.render('images/interactions.gv', view=False)  
    display(Image('images/interactions.gv.png'))
    return

def create_distance_network(coordinates_df, max_dist=100, min_seconds=3):
    """This function takes as parameters the optional coordinates file, the maximal distance for two fish 
    in a frame for a edge to be drawn between them, and assuming 25 fps, the minimum amount of seconds which
    the two fish have to be in the specified distance. 
    A loop is checking for each frame the distance from each fish to each fish, that is the reason why the output
    may take a few seconds until loaded."""
    #get the range of the frames (i.e. 25/second) to loop through
    first_frame = coordinates_df.frame.min()
    last_frame = coordinates_df.frame.max()
    frames_list = list(range(first_frame, last_frame,5))
    close_fish_list = []
    #take a slice of coordinates_df for each frame and calculate the 
    #distances from each fish to each other fish
    for frame in frames_list:
        frame_df = coordinates_df[coordinates_df.frame == frame]
        i=0
        while i < len(frame_df)-1:
            k=i+1
            while k < len(frame_df):
                #calculate the distance for fish k and fish i
                x = abs(frame_df.x.iloc[i] - frame_df.x.iloc[k])
                y = abs(frame_df.y.iloc[i] - frame_df.y.iloc[k])
                dist = math.sqrt(x**2 + y**2)
                #add an entry to the close_fish_list if dist < threshold
                if dist <= max_dist:
                    close_fish_list.append((frame_df.id.iloc[i], frame_df.id.iloc[k]))
                k+=1
            i+=1
    #create edges and attributes for network generation
    edges_df = pd.DataFrame(close_fish_list, columns=['fish_1', 'fish_2'])
    edges_df = edges_df.groupby(edges_df.columns.tolist(), as_index=False).size().to_frame(name='frames').reset_index()
    edges_df['tuples'] = list(zip(edges_df.fish_1, edges_df.fish_2))
    edges_df['close_seconds'] = edges_df.apply(lambda row: row.frames / 5, axis=1)
    edges_df = edges_df[edges_df.close_seconds >= min_seconds]
    edge_attributes_label = dict(zip(edges_df.tuples, edges_df.close_seconds))
    #change for edge weight
    edges_df.close_seconds = edges_df.close_seconds * 3 / edges_df.close_seconds.max()
    edge_attributes_weight = dict(zip(edges_df.tuples, edges_df.close_seconds))
    #create undirected graph with attributes
    G = nx.Graph()
    G.add_edges_from(edges_df.tuples)
    nx.set_edge_attributes(G, edge_attributes_label, name='label')
    nx.set_edge_attributes(G, edge_attributes_weight, name='penwidth')
    #graphviz
    G_dot_string = to_pydot(G).to_string()
    G_dot = Source(G_dot_string)
    G_dot.format= 'png'
    G_dot.render('images/distances.gv', view=False)  
    display(Image('images/distances.gv.png'))
    return 

def create_trajectory_map(coordinates_df, boris_df):
    """Input parameters are the behavior and the coordinates file, the behavior file is used for the IDs, 
    so the colour scheme of the trajectories taken from the coordinates file is consistent with the colors
    from the plots. The trajectories are done by scattering the x- and y- coordinates for each fish for each
    frame together in one plot."""
    #id works only for jakobs positions
    fish_ids = get_fish_ids(boris_df)
    #fish_ids = coordinates_df.id.unique().tolist()
    trajectory_list = []
    fig = plt.figure(figsize=(9,7))
    ax = fig.subplots()
    for fish in fish_ids:
        #extract positions for the fish and scatter it
        coordinates = coordinates_df[coordinates_df.id==fish]
        trajectory = ax.scatter(coordinates.x, coordinates.y, 0.1)
        trajectory_list.append(trajectory)
    plt.legend(trajectory_list, fish_ids, markerscale=20)   
    plt.xlabel("x-coordinate", fontsize=18, labelpad=10)
    plt.ylabel("y-coordinate", fontsize=18, labelpad=10)
    fig.savefig('images/trajectory_map.png', bbox_inches='tight')
    return plt


def create_accumulate_actions_plot(boris_df, behavior):
    """Input parameter is the behavior file, the output is a static view of a accumulation of all actions 
    of all IDs accumulated over time, so the total amount is viewable as well as when the number of actions 
    increased most."""
    #ugly programming follows but I could not solve otherwise in a couple hours so I was exhausted and made it ugly
    fish_ids = get_fish_ids(boris_df)
    fig = plt.figure(figsize=(9,7))
    for fish in fish_ids:
        fish_df = boris_df[boris_df.subject == fish] 
        if 'behavioral_category' in boris_df:
            categories = fish_df[fish_df.behavioral_category == behavior]
            behaviors = fish_df[fish_df.behavior == behavior]
            fish_df = categories.append(behaviors)
        else:
            fish_df = fish_df[fish_df.behavior == behavior]
        sum_of_rows = range(1,len(fish_df)+1)
        plt.plot(fish_df.time, sum_of_rows, label=fish)
        #plt.plot([fish_df.time.max(),boris_df.time.max()], [len(fish_df),len(fish_df)])
    plt.gca().set_prop_cycle(None)
    for fish in fish_ids:
        fish_df = boris_df[boris_df.subject == fish]
        if 'behavioral_category' in boris_df:
            categories = fish_df[fish_df.behavioral_category == behavior]
            behaviors = fish_df[fish_df.behavior == behavior]
            fish_df = categories.append(behaviors)
        else:
            fish_df = fish_df[fish_df.behavior == behavior]
        plt.plot([fish_df.time.max(),boris_df.time.max()], [len(fish_df),len(fish_df)])
    plt.gca().set_prop_cycle(None)
    for fish in fish_ids:
        fish_df = boris_df[boris_df.subject == fish]
        if 'behavioral_category' in boris_df:
            categories = fish_df[fish_df.behavioral_category == behavior]
            behaviors = fish_df[fish_df.behavior == behavior]
            fish_df = categories.append(behaviors)
        else:
            fish_df = fish_df[fish_df.behavior == behavior]
        plt.plot([0,fish_df.time.min()], [0,1])
    plt.legend()
    plt.xlabel("time in s", fontsize=18, labelpad=10)
    plt.ylabel("#behaviors", fontsize=18, labelpad=10)
    plt.show()
    fig.savefig('images/accumulate_actions_plot.png', bbox_inches='tight')
    return plt

def create_activity_plot(boris_df, intervals=10):
    """Input parameters are the behavior file and the number of intervals the user wants to have displayed.
    Basically it is the same like the accumulate_actions_plot, but here the user can specify the intervals so 
    some correlations may be better to see."""
    fish_ids = get_fish_ids(boris_df)
    #retrieve interval size from amount of time intervals and the 
    #maximum value of boris_df.time
    max_time = boris_df.time.max().astype(int)
    interval_size = (max_time / intervals).astype(int)
    interval_list = range(interval_size, max_time+interval_size, interval_size)
    fig = plt.figure(figsize=(9,7))
    for fish in fish_ids:
        sum_actions=[]
        fish_df = boris_df[boris_df.subject == fish]
        for interval in interval_list:
            actions = len(fish_df[fish_df.time.astype(int) <= interval])
            sum_actions.append(actions)
        #take the differences and insert the first value again
        res = [sum_actions[i+1] - sum_actions[i] for i in range(len(sum_actions)-1)]
        res.insert(0,sum_actions[0])
        plt.plot(interval_list, res, label=fish)
    plt.legend()
    plt.xlabel("time in s", fontsize=16, labelpad=10)
    plt.ylabel("#behaviors in interval", fontsize=16, labelpad=10)
    

def create_dashboard():
    #handle user inputed files correctly by checking if it is an .xlsx or .csv file
    try:
        [behavior] = uploader_bhvr.value
    except: 
        print('You need to upload an behavior file first.')
        return
    try:
        boris_df = clean_boris_df(pd.read_csv(io.BytesIO(uploader_bhvr.value[behavior]["content"])))
    except:
        boris_df = clean_boris_df(pd.read_excel(io.BytesIO(uploader_bhvr.value[behavior]["content"])))
    #try using the optional coordinates file, fails if file is not uploaded
    coordinates_present = True
    try:
        [coordinates] = uploader_pos.value
        try:
            coordinates_df = pd.read_csv(io.BytesIO(uploader_pos.value[coordinates]["content"]))
        except: 
            coordinates_df = pd.read_excel(io.BytesIO(uploader_pos.value[coordinates]["content"]))
        coordinates_df.columns = [x.lower() for x in coordinates_df.columns]
    except: 
        coordinates_present = False
        display(Markdown("""#### Trajectory Map and Distance network cannot be computed as trajectorie-file is not uploaded."""))
    #display metainformation for the user
    display(Markdown("""### IDs"""))
    print(get_fish_ids(boris_df))
    if 'behavioral_category' in boris_df:
        display(Markdown("""### Behavioral categories"""))
        print(boris_df.behavioral_category.unique())
    display(Markdown("""### Behaviors"""))
    print(boris_df.behavior.unique())
    #display transition probability network
    display(Markdown("""## Transition probability network \n ##### kind_of_cycle: Choose between behavior or behavioral category \n ##### min_count: Choose the minimum edge weight for an edge to be displayed \n ##### to_remove: Type name of behavior to be removed of display. Changing 'kind_of_cycle' twice adds removed behaviors.\n ##### with_status: Include status of behaviors in network \n ##### normalized: Normalize sum of outgoing edges to 1 per node\n
    """))
    behavior_cycle = interactive(create_behavior_cycle, {'manual': True}, boris_df = fixed(boris_df), kind_of_cycle=['behavior', 'behavioral_category'], min_count=(0,1,0.02), rmv_id='', rmv_bhvr='', with_status=False, normalized=True)
    display(behavior_cycle) 
    if coordinates_present:
        display(Markdown("""## Trajectory map"""))
        trajectory_map = interactive(create_trajectory_map, coordinates_df = fixed(coordinates_df), boris_df = fixed(boris_df))
        display(trajectory_map)
        display(Markdown("""## Distance network \n Assuming we have 25 frames per second, the edge count is increased by 1/25 for each frame in which the distance between two fish is smaller than 'max_dist'. 
        \n The edge is displayed if the count is bigger than 'min_seconds'. The computation may take a few seconds if the dataset is large. \n
        PLEASE BE PATIENT, COMPUTATION TAKES 5 TO 10 SECONDS."""))
        distance_network = interactive(create_distance_network, coordinates_df = fixed(coordinates_df), max_dist = (10,500,5), min_seconds = (1,600,5))
        display(distance_network)
    #display interactive interaction network
    display(Markdown("""## Interaction network \n Nodes are subjects/objects of behaviors. \n Choose the minimum count of directed behaviors for an edge to be displayed"""))
    graph = interactive(create_interaction_network, boris_df = fixed(boris_df), min_interactions=(1,100,1))
    display(graph)
    #display behavior or behavioral category
    display(Markdown("""## Behaviors accumulated over time \n You choose the behavior or behavioral category to be displayed"""))
    if 'behavioral_category' in boris_df:
        accumulate_actions = interactive(create_accumulate_actions_plot, boris_df=fixed(boris_df), behavior = np.concatenate([boris_df.behavioral_category.unique(),boris_df.behavior.unique()]))
    else:
        accumulate_actions = interactive(create_accumulate_actions_plot, boris_df=fixed(boris_df), behavior = boris_df.behavior.unique())
    display(accumulate_actions)
    #display activity plot
    display(Markdown("""## Activity plot \n All behaviors are accumulated over time. You choose the size of the intervals."""))
    activity_plot = interactive(create_activity_plot, boris_df=fixed(boris_df), intervals = (1,100,1))
    display(activity_plot)
    return


def clean_boris_df(boris_df):
    """This function precleans the dataset by deleting header information if present. Furthermore
    it standardizes all column names."""
    #set all headers to lowercase
    boris_df.columns = [x.lower() for x in boris_df.columns]
    
    #for etiennes data
    if 'observation id' in boris_df.iloc[0]:
        boris_df.columns = ['time', 'media_file_path', 'total_length', 'fps', 'subject', 'behavior', 'behavioral_category', 'modifier_1', 'comment', 'status']
        boris_df = boris_df.dropna(axis=0, subset=['subject'])
        boris_df = boris_df.iloc[1:]
    
    #for jakobs data
    if 'modifier 1' in boris_df:
        boris_df.rename(columns = {'modifier 1':'modifier_1'}, inplace = True)
    if 'behavioral category' in boris_df:
        boris_df.rename(columns = {'behavioral category':'behavioral_category'}, inplace = True)
        
    #for huys data
    if 'subject' in boris_df and len(boris_df.subject.unique()) == 2:
        if 'modifier_1' not in boris_df:
            boris_df['modifier_1'] = 'Left'
            boris_df['modifier_1'] = np.where(boris_df['subject'] == 'Left', 'Right', boris_df['modifier_1'])
            boris_df['behavioral_category'] = 'restrained aggression'
            boris_df['behavioral_category'] = np.where(boris_df['behavior'] == 'bite', 'overt aggression', boris_df['behavioral_category'])
            boris_df['behavioral_category'] = np.where(boris_df['behavior'] == 'mouth', 'overt aggression', boris_df['behavioral_category'])
    
    #convert time to float if excel gives string objects   
    boris_df.time = boris_df.time.astype(float)
    return boris_df


global remove_list_cat
remove_list_cat = []
global remove_list
remove_list = []
global remove_id_list
remove_id_list = []
def create_behavior_cycle(boris_df, kind_of_cycle, min_count, rmv_id, rmv_bhvr, with_status, normalized):
    """Input parameters are the behavior file, the specification if the user wants to see the behaviors itself 
    or the behavior cycle of the behavioral categories and the minimal count for a edge to be displayed. 
    This cycle is calculated by splitting the boris-file for each fish and then increasing the edge count for each 
    successing behavior. In the end, the edge count is normalized in [0,1] for each node where edges come from 
    so we have kind of a probability of which behavior follows which behavior"""
    fish_ids = get_fish_ids(boris_df)
    successor_list = []
    #prepare dataframe with user input
    #first check if the user wants so see the behaviors or the behavioral categories
    if kind_of_cycle == 'behavioral_category':
        #reset list of removed behaviors
        remove_list.clear()
        boris_df['chosen_data'] = boris_df['behavioral_category']
        display(Markdown("""#### All behavioral categories: \n"""))
        
        print(boris_df.chosen_data.unique())
        if rmv_bhvr:
            remove_us = rmv_bhvr.split(',')
            for x in remove_us:
                remove_list_cat.append(x)
        if remove_list_cat:
            display(Markdown("""#### Removed behavioral categories: \n"""))
            print(set(remove_list_cat))
            for x in remove_list_cat:
                boris_df = boris_df.drop(boris_df[boris_df.chosen_data == x].index)
        else: 
            display(Markdown("""#### No behavioral categories removed yet: \n"""))
    else:
        #reset list of removed behavioral categories
        remove_list_cat.clear()
        boris_df['chosen_data'] = boris_df['behavior']
        display(Markdown("""#### All behaviors: \n"""))
        print(boris_df.chosen_data.unique())
        if rmv_bhvr:
            remove_us2 = rmv_bhvr.split(',')
            for x in remove_us2:
                remove_list.append(x)
        if remove_list:
            display(Markdown("""#### Removed behavior: \n"""))
            print(set(remove_list))
            for x in remove_list:
                boris_df = boris_df.drop(boris_df[boris_df.chosen_data == x].index)
        else: 
            display(Markdown("""#### No behaviors removed yet \n"""))
   
    #remove IDs
    if rmv_id:
        remove_ids = rmv_id.split(',')
        for x in remove_ids:
            remove_id_list.append(x)
    if remove_id_list:
        display(Markdown("""#### Removed IDs: \n"""))
        print(set(remove_id_list))
    fish_ids_after_removal = [x for x in fish_ids if x not in remove_id_list]
    if not fish_ids_after_removal:
        print("It does not work if you remove all IDs my friend :)")
            
    
   
    #loop through dataframe for each fish and add behavior and successor
    for fish in fish_ids_after_removal:
        id_frame = boris_df[boris_df.subject == fish]  
        if not (with_status):
            id_frame = id_frame.drop(id_frame[id_frame.status == 'STOP'].index)
        i=0
        k=i+1
        while i < len(id_frame)-1:
            successor_list.append((id_frame.chosen_data.iloc[i], id_frame.status.iloc[i], id_frame.chosen_data.iloc[k], id_frame.status.iloc[k]))
            k+=1
            i+=1
    #lets make an edgelist with behavior and successor
    successor_df = pd.DataFrame(successor_list, columns=['action_1', 'status_1', 'action_2', 'status_2'])
    if (with_status):
        successor_df['action_1'] = successor_df['action_1'] + ' ' + successor_df['status_1']
        successor_df['action_2'] = successor_df['action_2'] + ' ' + successor_df['status_2']
    else:
        successor_df = successor_df.replace(to_replace="POINT", value="")
        
    successor_df['tuples'] = list(zip(successor_df.action_1, successor_df.action_2))
    successor_df = successor_df.groupby(successor_df.columns.tolist(), as_index=False).size().to_frame(name='records').reset_index()

    #normalize the records in [0,1] so that all together are 1 for each action
    behavior_ids = successor_df.action_1.unique().tolist()
    edges_df = pd.DataFrame()
    for action in behavior_ids:
        action_frame = successor_df[successor_df.action_1 == action]
        if(normalized):    
            sum_of_successors = action_frame.records.sum()
            action_frame.records = action_frame.records.div(sum_of_successors).round(2)
        edges_df = edges_df.append(action_frame)   
    edges_df = edges_df[edges_df.records > min_count]
    #create directed graph with edges and edge attributes
    G = nx.DiGraph()
    G.add_edges_from(edges_df.tuples)
    edge_attributes_label = dict(zip(edges_df.tuples, edges_df.records))
    #change edges records for weight
    if(normalized):
        edges_df.records = edges_df.records * 3
        edge_attributes_weight = dict(zip(edges_df.tuples, edges_df.records))
    else:
        edge_attributes_weight = dict(zip(edges_df.tuples, 3*(edges_df.records/edges_df.records.max())))
    nx.set_edge_attributes(G, edge_attributes_weight, name='penwidth')
    nx.set_edge_attributes(G, edge_attributes_label, name='label')
    #graphviz
    G_dot_string = to_pydot(G).to_string()
    G_dot = Source(G_dot_string)
    G_dot.format= 'png'
    G_dot.render('images/transitions.gv', view=False)  
    display(Image('images/transitions.gv.png'))
    return 
    

In [51]:
display(Markdown("""# BehaviorAnalyzer \n
#### Use this interactive tool to visually analyze your behavior data of animals. \n
Your provided data should have the following structure ['time', 'total_length', 'fps', 'subject', 'behavior', 'behavioral_category', 'modifier_1', 'status']. \n
If your data includes only two individuals you may omit 'modifier_1'. File formats may be 'csv' or 'xlsx'. \n
Optionally you can upload a corresponding coordinates/trajectories file with the structure ['row_number', 'id', 'frame', 'x', 'y'].\n"""))
display(Markdown(""" #### Please click the 'Behavior'-button to upload your behavior file . Then click on 'Analyse data!'."""))


# BehaviorAnalyzer 

#### Use this interactive tool to visually analyze your behavior data of animals. 

Your provided data should have the following structure ['time', 'total_length', 'fps', 'subject', 'behavior', 'behavioral_category', 'modifier_1', 'status']. 

If your data includes only two individuals you may omit 'modifier_1'. File formats may be 'csv' or 'xlsx'. 

Optionally you can upload a corresponding coordinates/trajectories file with the structure ['row_number', 'id', 'frame', 'x', 'y'].


 #### Please click the 'Behavior'-button to upload your behavior file . Then click on 'Analyse data!'.

In [12]:
#buttons

#reset user input
reset_button = widgets.Button(description="Clear data")
def reset(_):
    with out:
        out.clear_output()
    #uploader_bhvr.close()
    #uploader_pos.close()
reset_button.on_click(reset)
display(reset_button)

#upload behavior and trajectories
uploader_bhvr = widgets.FileUpload(description='Behavior', multiple=True)
display(uploader_bhvr)
uploader_pos = widgets.FileUpload(description='Trajectories', multiple=True)
display(uploader_pos)

#analyze uploaded data
analyse_button = widgets.Button(description="Analyse data!")
out = widgets.Output()
def analyse(_):
      # "linking function with output"
      with out:
        # what happens when we press the button
        out.clear_output()
        create_dashboard()
analyse_button.on_click(analyse)
# displaying button and its output together
widgets.VBox([analyse_button,out])


Button(description='Clear data', style=ButtonStyle())

FileUpload(value={}, description='Behavior', multiple=True)

FileUpload(value={}, description='Trajectories', multiple=True)

In [53]:
display(Markdown("""  This tool was developed at the University of Constance under supervision of Michael Aichem and Dr. Karsten Klein from the laboratory for Computational Life Sciences. Valuable feedback and data was provided by Etienne Lein, Manh Huy Nguyen, Jakob Guebel and Dr. Alex Jordan from the laboratory for the Evolution of Collective and Social Behavior. \n
The tool is written in Python, using 'networkx' for network generation, 'GraphViz' for drawing and 'voila' in combination with 'heroku' for deploying.  \n
Please send bugs or recommendations to nicolai.kraus@uni-konstanz.de."""))


  This tool was developed at the University of Constance under supervision of Michael Aichem and Dr. Karsten Klein from the laboratory for Computational Life Sciences. Valuable feedback and data was provided by Etienne Lein, Manh Huy Nguyen, Jakob Guebel and Dr. Alex Jordan from the laboratory for the Evolution of Collective and Social Behavior. 

The tool is written in Python, using 'networkx' for network generation, 'GraphViz' for drawing and 'voila' in combination with 'heroku' for deploying.  

Please send bugs or recommendations to nicolai.kraus@uni-konstanz.de.